# Azure AI Agents with Model Context Protocol (MCP) Support - Python

This notebook demonstrates how to use Azure AI Agents with Model Context Protocol (MCP) tools in Python. It shows how to create an intelligent agent that can leverage external MCP servers (like Microsoft Learn) for enhanced capabilities using keyless authentication.

## Install Required Python Packages

First, we need to install the necessary Python packages:
- **azure-ai-projects**: Core Azure AI Projects SDK
- **azure-ai-agents**: Azure AI Agents SDK for creating and managing agents
- **azure-identity**: Provides keyless authentication using DefaultAzureCredential
- **mcp**: Model Context Protocol implementation for Python

## Keyless Authentication Benefits

This notebook demonstrates **keyless authentication** which provides several advantages:
- ✅ **No API keys to manage** - Uses Azure identity-based authentication
- ✅ **Enhanced security** - No secrets stored in code or configuration files
- ✅ **Automatic credential rotation** - Azure handles credential lifecycle management
- ✅ **Role-based access control** - Uses Azure RBAC for fine-grained permissions
- ✅ **Multi-environment support** - Works seamlessly across development and production

The `DefaultAzureCredential` automatically selects the best available credential source:
1. **Managed Identity** (when running in Azure)
2. **Azure CLI** credentials (during local development)
3. **Visual Studio** credentials
4. **Environment variables** (if configured)
5. **Interactive browser** authentication (as fallback)

## Keyless Authentication Setup

**Prerequisites for keyless authentication:**

### For Local Development:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### For Azure Environments:
- Enable **System-assigned Managed Identity** on your Azure resource
- Assign appropriate **RBAC roles** to the managed identity:
  - `Cognitive Services OpenAI User` for Azure OpenAI access
  - `AI Developer` for Azure AI Projects access

### Environment Variables (Optional):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**No API keys or connection strings needed!** 🔐

In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Import Required Libraries

Import the necessary Python modules:
- **os, time**: Standard Python libraries for environment variables and delays
- **AIProjectClient**: Main client for Azure AI Projects
- **DefaultAzureCredential**: Keyless authentication for Azure services
- **MCP-related classes**: For creating and managing MCP tools and handling approvals

In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Configure MCP Server Settings

Set up the MCP server configuration using environment variables with fallback defaults:
- **MCP_SERVER_URL**: The URL of the MCP server (defaults to Microsoft Learn API)
- **MCP_SERVER_LABEL**: A label to identify the MCP server (defaults to "mslearn")

This approach allows for flexible configuration across different environments.

In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Create Azure AI Project Client (Keyless Authentication)

Initialize the Azure AI Project client using **keyless authentication**:
- **endpoint**: The Azure AI Foundry project endpoint URL
- **credential**: Uses `DefaultAzureCredential()` for secure, keyless authentication
- **No API keys required**: Automatically discovers and uses the best available credential

**Authentication Flow:**
1. Checks for Managed Identity (in Azure environments)
2. Falls back to Azure CLI credentials (for local development)
3. Uses other available credential sources as needed

This approach eliminates the need to manage API keys or connection strings in your code.

In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Create MCP Tool Definition

Create an MCP tool that connects to the Microsoft Learn MCP server:
- **server_label**: Identifier for the MCP server
- **server_url**: URL endpoint of the MCP server
- **allowed_tools**: Optional list to restrict which tools can be used (empty list allows all tools)

This tool will enable the agent to access Microsoft Learn documentation and resources.

In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Create Agent and Execute Conversation (Keyless Workflow)

This comprehensive section demonstrates the complete **keyless agent workflow**:

1. **Create AI Agent**: Set up an agent with GPT-4.1 nano model and MCP tools
2. **Create Thread**: Establish a conversation thread for communication
3. **Send Message**: Ask the agent about Azure OpenAI vs OpenAI differences
4. **Handle Tool Approvals**: Automatically approve MCP tool calls when required
5. **Monitor Execution**: Track the agent's progress and handle any required actions
6. **Display Results**: Show the conversation and tool usage details

**Keyless Features:**
- ✅ **No hardcoded secrets** - All authentication handled by Azure identity
- ✅ **Secure by default** - Uses role-based access control
- ✅ **Simplified deployment** - No credential management required
- ✅ **Audit-friendly** - All access is tracked through Azure identity

The agent will use MCP tools to access Microsoft Learn resources with full security and no API key management.

In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")